<a href="https://colab.research.google.com/github/saulolvieira/project_1_ws_ba/blob/main/ba_data_input_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importando as bibliotecas

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup


URL do site indicado: "https://www.airlinequality.com/airline-reviews/british-airways/page/1/?sortby=post_date%3ADesc&pagesize=100"

# Funções

In [ ]:
# Função para buscar as informações básicas de cada comentário ( nota, comentário e data)
def get_basic(url):
  response = requests.get(url)
  soup = BeautifulSoup(response.content, 'html.parser')
  comments = soup.find_all('article', attrs={'itemprop': 'review'})

  # Buscando informações básicas
  data_basic = []
  for comment in comments:
      row = {}
      row['review_id'] = comment.find('div', class_='body')['id']
      row['date'] = comment.find('meta')['content']
      score_elem = comment.find('span', attrs={'itemprop': 'ratingValue'})
      if score_elem:
        row['score'] = score_elem.text
      else:
        row['score'] = 0
      row['resume'] = comment.find('h2', class_='text_header').text
      row['review'] = comment.find('div', class_='text_content').text
      data_basic.append(row)

  df_basic = pd.DataFrame(data_basic)
  return df_basic

In [ ]:
# Função para buscar todas as informações analíticas de cada comentário de cada página (tabela depois do comentário)

def get_analytical(url):
  response = requests.get(url)
  soup = BeautifulSoup(response.content, 'html.parser')
  comments = soup.find_all('article', attrs={'itemprop': 'review'})

  # Buscando informações básicas


  # Encontrando todos os ids

  id_ = []
  for comment in comments:
    row = {}
    row['id'] = comment.find('div', class_='body')['id']
    id_.append(row)

  df_id = pd.DataFrame(id_)


  # Encontrar todos os parâmetros de todos os id's do dataframe:

  df_results = pd.DataFrame()

  for i in df_id['id']:
    # Conteúdo da página para cada id
    div = soup.find('div', {'class': 'body', 'id': i})
    div = BeautifulSoup(str(div), 'html.parser')

    # Listas vazias para adicionar os valores de cada coluna
    titles = []
    values = []
    review_id = i

    # encontre todas as tags <tr> de cada elemento da tabela
    rows = [element.find_all('tr') for element in div]
    # desempacote a lista de listas em uma única lista
    rows = [tr for sublist in rows for tr in sublist]

  # extrair títulos e valores da tabela
    for row in rows:
        title_element = row.find('td', {'class': 'review-rating-header'})
        value_element = row.find('td', {'class': 'review-value'})

        if title_element is not None and value_element is not None:
            title = title_element.text.strip()
            value = value_element.text.strip()
            titles.append(title)
            values.append(value)

        # caso exista a classe review-rating-stars, também é possível coletar as informações
        rating_stars = row.find('td', {'class': 'review-rating-stars'})
        if rating_stars is not None:
            title = title_element.text.strip()
            star_spans = rating_stars.find_all('span', {'class': 'star'})
            star_fill_spans = [span for span in star_spans if 'fill' in span['class']]
            value = str(len(star_fill_spans))
            titles.append(title)
            values.append(value)

    # criar um dicionário com as listas de títulos e valores
    data = {'review_id': review_id, 'titles': titles, 'values': values}
    df = pd.DataFrame(data)
    df_results = pd.concat([df_results, df])

  df_analytical = df_results.pivot(index='review_id', columns='titles', values='values')

  return df_analytical

# Iterações

In [ ]:
#Buscando as infomações analíticas de todos os comentários da página
# define a quantidade de páginas a serem percorridas
num_pages = 36
all_results = []
# loop para chamar a função para cada página
for i in range(1, num_pages+1):
    url = f"https://www.airlinequality.com/airline-reviews/british-airways/page/{i}/?sortby=post_date%3ADesc&pagesize=100"
    scraps = get_analytical(url)
    all_results.append(scraps)
# concatena todos os dataframes em um único dataframe
df = pd.concat(all_results)
df = df.reset_index()

In [ ]:
#Buscando as infomações básicas de todos os comentários da página
num_pages = 36
all_results2 = []
# loop para chamar a função para cada página
for i in range(1, num_pages+1):
    url2 = f"https://www.airlinequality.com/airline-reviews/british-airways/page/{i}/?sortby=post_date%3ADesc&pagesize=100"
    scraps2 = get_basic(url2)
    all_results2.append(scraps2)
# concatena todos os dataframes em um único dataframe
df2 = pd.concat(all_results2)
df2 = df2.reset_index()

# Consolidação final

In [ ]:
df_final = pd.merge(df2, df, on = 'review_id', how = 'inner')